In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark=SparkSession.builder.appName("sql").master('local[*]').getOrCreate()

In [3]:
# joins can be done on two dataframes having similar columns. There are many types of joins. 
# join usually has the beow syntax
#  dataframe1.join(dataframe2,dataframre1.colname==dataframe2.colname,"type")
# this will return a new dataframe with data joined.

data1=[(1, "Shreyas"),(2,"jayaram")]
schema1=["Id","Name"]
data2=[(1,20),(3,10)]
schema2=["Id","Age"]
df1=spark.createDataFrame(data1,schema1)
df2=spark.createDataFrame(data2,schema2)
df1.show()
df2.show()

+---+-------+
| Id|   Name|
+---+-------+
|  1|Shreyas|
|  2|jayaram|
+---+-------+

+---+---+
| Id|Age|
+---+---+
|  1| 20|
|  3| 10|
+---+---+



In [4]:
df1.join(df2,df1.Id==df2.Id,"Inner").show()
df1.join(df2,df1.Id==df2.Id,"Outer").show()
df1.join(df2,df1.Id==df2.Id,"Left" ).show()  # Left_outer can aLso be used
df1.join(df2,df1.Id==df2.Id,"Right" ).show() # right_outer can aLso be used

+---+-------+---+---+
| Id|   Name| Id|Age|
+---+-------+---+---+
|  1|Shreyas|  1| 20|
+---+-------+---+---+

+----+-------+----+----+
|  Id|   Name|  Id| Age|
+----+-------+----+----+
|   1|Shreyas|   1|  20|
|   2|jayaram|NULL|NULL|
|NULL|   NULL|   3|  10|
+----+-------+----+----+

+---+-------+----+----+
| Id|   Name|  Id| Age|
+---+-------+----+----+
|  1|Shreyas|   1|  20|
|  2|jayaram|NULL|NULL|
+---+-------+----+----+

+----+-------+---+---+
|  Id|   Name| Id|Age|
+----+-------+---+---+
|   1|Shreyas|  1| 20|
|NULL|   NULL|  3| 10|
+----+-------+---+---+



In [5]:
# Now lets see semi and anti joins
df1.join(df2,df1.Id==df2.Id,"Left" ).show() # It will consider the data only from Left dataframe which did not match with other dataframe.
df1.join(df2,df1.Id==df2.Id,"Left" ).show() # opposite to semi join, It will consider the data only from Left dataframe which matches with other dataframe.

df1.crossJoin(df2).show() # It will join all rows in one datafrane with all rows in another. Here there is no join condition.


+---+-------+----+----+
| Id|   Name|  Id| Age|
+---+-------+----+----+
|  1|Shreyas|   1|  20|
|  2|jayaram|NULL|NULL|
+---+-------+----+----+

+---+-------+----+----+
| Id|   Name|  Id| Age|
+---+-------+----+----+
|  1|Shreyas|   1|  20|
|  2|jayaram|NULL|NULL|
+---+-------+----+----+

+---+-------+---+---+
| Id|   Name| Id|Age|
+---+-------+---+---+
|  1|Shreyas|  1| 20|
|  1|Shreyas|  3| 10|
|  2|jayaram|  1| 20|
|  2|jayaram|  3| 10|
+---+-------+---+---+



In [7]:
# Example of self join
#df1.alias("Table1").join(df1.alias("Table2"),col(Table1.Id)==col(Table2.Id),"Inner")
# As we are using same datafnane, we have to give alias name.

#Complications with join.
#one of the Complication with join is that when we join tables hav ng same column names for key columns, then output dataframe will have two coLunn names some.
# Then when we try to access that columnn, it will throw an error as there are two columns with same name.
#To resolve this we can either drop one of the column after joining or rename the columnn before joining.
from pyspark.sql.functions import *
df1.join(df2, df1.Id==df2.Id, "Inner").drop(df1.Id).select("Id").show()

df3=df1.withColumnRenamed("Id","Id_1")
df3.join(df2, df3.Id_1==df2.Id,"Inner").select("Id").show()


+---+
| Id|
+---+
|  1|
+---+

+---+
| Id|
+---+
|  1|
+---+

